In [36]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl

In [37]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


Chamando as bases

In [50]:
df = pd.read_excel('ESTADIC//Base_ESTADIC_2021.xlsx', sheet_name='Educação', usecols=['Cod UF','EEDU161A','EEDU161B'])
df

,Cod UF,EEDU161A,EEDU161B
0,11,680,2016
1,12,330,2017
2,13,3951,2013
3,14,892,2014
4,15,7442,2010
5,16,949,2015
6,17,2851,2014
7,21,9860,2013
8,22,71,2006
9,23,15575,2014


In [51]:
uf = pd.read_excel('ESTADIC//Base_ESTADIC_2018.xlsx', sheet_name = 'Variáveis externas', usecols=[1,2,3])
uf

,COD UF,UF,NOME UF
0,11,RO,RONDÔNIA
1,12,AC,ACRE
2,13,AM,AMAZONAS
3,14,RR,RORAIMA
4,15,PA,PARÁ
5,16,AP,AMAPÁ
6,17,TO,TOCANTINS
7,21,MA,MARANHÃO
8,22,PI,PIAUÍ
9,23,CE,CEARÁ


Renomeando as colunas

In [52]:
df.columns

Index(['Cod UF', 'EEDU161A', 'EEDU161B'], dtype='object')

In [53]:
df= df.rename(columns={'Cod UF':'cod_uf',
                        'EEDU161A':'lei_criacao_plano_carreira',
                        'EEDU161B':'ano_criacao_lei'}) 

In [54]:
##adicionando a coluna de 'ano' em cada um dos dfs
df['ano']=2021 

In [55]:
x= uf.pivot_table(columns=('COD UF', 'UF', 'NOME UF'), aggfunc='size')


In [56]:
uf = pd.DataFrame(x).reset_index()[['COD UF', 'UF', 'NOME UF']]

In [57]:
df = df.merge(uf, right_on='COD UF',left_on='cod_uf') #adicionando sigla e nome das UFs


In [58]:
df = df.drop(['COD UF'], axis=1) #eliminando coluna repetida

In [59]:
df= df.rename(columns={'UF':'sigla_uf',
                'NOME UF':'uf'})

In [62]:
df

,cod_uf,lei_criacao_plano_carreira,ano_criacao_lei,ano,sigla_uf,uf
0,11,680,2016,2021,RO,Rondônia
1,12,330,2017,2021,AC,Acre
2,13,3951,2013,2021,AM,Amazonas
3,14,892,2014,2021,RR,Roraima
4,15,7442,2010,2021,PA,Pará
5,16,949,2015,2021,AP,Amapá
6,17,2851,2014,2021,TO,Tocantins
7,21,9860,2013,2021,MA,Maranhão
8,22,71,2006,2021,PI,Piauí
9,23,15575,2014,2021,CE,Ceará


In [61]:
df['uf'] = df['uf'].str.title()

In [63]:
df = df[['ano','cod_uf','sigla_uf','uf','ano_criacao_lei','lei_criacao_plano_carreira']] #organizando as colunas

In [64]:
df.columns

Index(['ano', 'cod_uf', 'sigla_uf', 'uf', 'ano_criacao_lei',
       'lei_criacao_plano_carreira'],
      dtype='object')

In [65]:
df['ano_criacao_lei'].unique()

array([2016, 2017, 2013, 2014, 2010, 2015, 2006, 2003, 1998, 2021, 2001,
       2002, 2004, 1990, 1997, 2020, 2000], dtype=int64)

In [67]:
limites = [0,2000, 2010, 2020, 9999] #criando uma nova coluna (periodo_anual) com base na coluna 'ano_criacao_lei'
categorias = ['Década de 90', 'Década de 2000', "Década de 2010", "Década de 2020" ]

df['periodo_anual'] = pd.cut(df['ano_criacao_lei'], bins=limites, labels=categorias)

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_11808\4053191972.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['periodo_anual'] = pd.cut(df['ano_criacao_lei'], bins=limites, labels=categorias)


In [68]:
df

,ano,cod_uf,sigla_uf,uf,ano_criacao_lei,lei_criacao_plano_carreira,periodo_anual
0,2021,11,RO,Rondônia,2016,680,Década de 2010
1,2021,12,AC,Acre,2017,330,Década de 2010
2,2021,13,AM,Amazonas,2013,3951,Década de 2010
3,2021,14,RR,Roraima,2014,892,Década de 2010
4,2021,15,PA,Pará,2010,7442,Década de 2000
5,2021,16,AP,Amapá,2015,949,Década de 2010
6,2021,17,TO,Tocantins,2014,2851,Década de 2010
7,2021,21,MA,Maranhão,2013,9860,Década de 2010
8,2021,22,PI,Piauí,2006,71,Década de 2000
9,2021,23,CE,Ceará,2014,15575,Década de 2010


In [69]:
df['periodo_anual'].unique()

['Década de 2010', 'Década de 2000', 'Década de 90', 'Década de 2020']
Categories (4, object): ['Década de 90' < 'Década de 2000' < 'Década de 2010' < 'Década de 2020']

In [30]:
df= df[['ano', 'sigla_uf','cod_uf', 'uf', 'ano_criacao_lei','periodo_anual', 'lei_criacao_plano_carreira']]

Subindo para o GBQ

In [70]:
client = bigquery.Client()
dataset_ref = client.dataset('estrutura_organizacional_carreiras')

In [71]:
df.dtypes

ano                              int64
cod_uf                           int64
sigla_uf                        object
uf                              object
ano_criacao_lei                  int64
lei_criacao_plano_carreira       int64
periodo_anual                 category
dtype: object

In [72]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
        bigquery.SchemaField('cod_uf','INTEGER',description='Código do IBGE da UF'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da UF'),
        bigquery.SchemaField('uf','STRING',description='Nome da UF'),        
        bigquery.SchemaField('ano_criacao_lei','INTEGER',description='Se a nomeação de diretor de escola ocorreu por concurso público '),
        bigquery.SchemaField('periodo_anual','STRING',description='Período anual referente a observação'),
        bigquery.SchemaField('lei_criacao_plano_carreira','INTEGER',description='Se a nomeação de diretor de escola ocorreu por indicação')
        ]


In [73]:
table_ref = dataset_ref.table('ESTADIC_lei_plano_carreira_por_ano')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=8164c25d-7324-4efe-bc14-cedf39f044f5>